In [1]:
from growing_instability_lib import *

Using TensorFlow backend.


In [2]:
sample_sub = pd.read_csv('../data/sampleSubmission.csv')
topics = sorted(set(sample_sub.columns.difference(['id'])))

topic2actual = {}
for i in sample_sub.columns:
    if 'id' == i:
        continue
    topic2actual[i] = segment(i)
    
target_columns = sorted(topics)
len(topics)

160

In [5]:
%%time

# wvec_trainingX = pd.read_hdf('training_data_wv_fs_no_stopwords', 'wvec_trainingX')
# fvec_trainingX = pd.read_hdf('training_data_wv_fs_no_stopwords', 'fvec_trainingX')
word2idx_trainingX = pd.read_hdf('training_data_wv_fs_no_stopwords', 'word2idx_trainingX')
# _word2idx = pd.read_hdf('training_data_wv_fs_no_stopwords', '_word2idx')
# trainingY = pd.read_hdf('training_data_wv_fs_no_stopwords', 'trainingY_fasttext')

CPU times: user 23.9 s, sys: 3.56 s, total: 27.5 s
Wall time: 32 s


In [13]:
len(word2idx_trainingX[1])

4775

In [8]:
fvec_trainingX

1890288

In [10]:
x = trainingY.sum(axis=1)
x[x.isnull()]

2005a_TrainingData_50466   NaN
1999b_TrainingData_26958   NaN
2003a_TrainingData_47804   NaN
1999a_TrainingData_25096   NaN
2012a_TrainingData_55452   NaN
2004a_TrainingData_48954   NaN
2008a_TrainingData_65998   NaN
2013a_TrainingData_52734   NaN
2000b_TrainingData_35244   NaN
2001a_TrainingData_40325   NaN
2005b_TrainingData_49338   NaN
2001b_TrainingData_44132   NaN
2010b_TrainingData_55071   NaN
2002b_TrainingData_45159   NaN
2006a_TrainingData_52970   NaN
2014b_TrainingData_52907   NaN
2006b_TrainingData_54181   NaN
2007b_TrainingData_61985   NaN
2007a_TrainingData_60094   NaN
2010a_TrainingData_44129   NaN
dtype: float64

In [7]:
ddf = df.ix[trainingY.index]
parsed_body = transform_text(ddf)

parsed_body_tokens = parsed_body.str.split(' ')
words_per_doc = parsed_body_tokens.map(len)

In [22]:
len([idx2word.get(i, ' ') for i in word2idx_trainingX.head().iloc[1]])

4775

In [9]:
vocabulary = set()

for b in parsed_body_tokens:
    vocabulary.update(b)

In [18]:
idx2word = {i: j for j, i in _word2idx.to_dict().items()}

In [10]:
dataset = parsed_body

ind2word = {i + 1: j for i, j in enumerate(vocabulary)}
ind2class = dict(enumerate(topics))

class2ind = {j: i for i, j in ind2class.items()}
word2ind = {j: i for i, j in ind2word.items()}

trainingX = parsed_body_tokens.map(lambda x: [word2ind[i] for i in x])
trainingY = ddf.topics.map(lambda x: [class2ind[i] for i in x if i in class2ind])

dataset = zip(trainingX, trainingY)

In [11]:
def generate_lstm_batch_dataset(dataset, word2ind, class2ind, max_len, batch_size=1000, shuffle=True):
    if shuffle:
        np.random.shuffle(dataset)

    num_docs = len(dataset)
    num_words = len(word2ind) + 1
    num_class = len(class2ind)

    for s in xrange(0, num_docs, batch_size):
        x_batch = np.zeros([batch_size, max_len, num_words])
        y_batch = np.zeros([batch_size, num_class])

        for ix, (features, target) in enumerate(dataset[s:s + batch_size]):
            # print features
            for idx, feat in enumerate(features):
                if idx >= max_len:
                    break

                # print feat, ind2word[feat]
                x_batch[ix, idx, feat] = 1

            if not isinstance(target, list):
                target = [target]
                
            for tg in target:
                y_batch[ix, tg] = 1

        yield x_batch[:ix + 1, :, :], y_batch[:ix + 1, :]


def infinite_lstm_dataset_generator(dataset, word2ind, class2ind, max_len, batch_size=100):
    while 1:
        for b in generate_lstm_batch_dataset(dataset, word2ind, class2ind, max_len, batch_size):
            yield b

# lens = []
# for i in dataset:
#     lens.append(len(i[0]))
# pd.Series(lens).quantile(0.999)
# Use the above to estimate the acceptable timeseries dimension.
LSTM_TIMESERIES = 100
id_lstm_gen = infinite_lstm_dataset_generator(dataset, word2ind, class2ind, max_len=LSTM_TIMESERIES, batch_size=10)



In [12]:
len(vocabulary)
max(ind2word)

83408

In [13]:
from keras.preprocessing import sequence

maxlen = 700
x_train = sequence.pad_sequences(trainingX, maxlen=maxlen)

def build_target(y, size):
    e = np.zeros(size)
    e[y] = 1
    return e


In [103]:
# Setup model
model_lstm = keras.models.Sequential()
model_lstm.add(keras.layers.Embedding(len(word2ind) + 1, 256))
# model_lstm.add(keras.layers.LSTM(32, return_sequences=False, input_shape=(None, len(word2ind) + 1)))
# model_lstm.add(keras.layers.Dropout(0.2))
model_lstm.add(keras.layers.LSTM(16, return_sequences=False))
model_lstm.add(keras.layers.Dense(128))
model_lstm.add(keras.layers.Activation('relu'))
model_lstm.add(keras.layers.Dropout(0.2))
model_lstm.add(keras.layers.Dense(len(class2ind)))
model_lstm.add(keras.layers.Activation('sigmoid'))
model_lstm.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# for i in range(6):
#     model_lstm.fit_generator(id_lstm_gen, steps_per_epoch=len(dataset), epochs=1)

In [104]:
w = np.vstack(trainingY.map(lambda x: build_target(x, len(topics)))).sum(axis=0) + 1
W = 1 / w

In [105]:
model_lstm.fit(
    x_train, np.vstack(trainingY.map(lambda x: build_target(x, len(topics)))),
    batch_size=1000,
    epochs=10,
#     class_weight=W
)

Epoch 1/10
9424/9424 [==============================] - 7s - loss: 0.6876 - acc: 0.7456     
Epoch 2/10
9424/9424 [==============================] - 7s - loss: 0.6455 - acc: 0.9165     
Epoch 3/10
9424/9424 [==============================] - 7s - loss: 0.5017 - acc: 0.9832     
Epoch 4/10
9424/9424 [==============================] - 7s - loss: 0.2621 - acc: 0.9904     
Epoch 5/10
9424/9424 [==============================] - 7s - loss: 0.0956 - acc: 0.9905     
Epoch 6/10
9424/9424 [==============================] - 7s - loss: 0.0556 - acc: 0.9905     
Epoch 7/10
9424/9424 [==============================] - 7s - loss: 0.0512 - acc: 0.9905     
Epoch 8/10
9424/9424 [==============================] - 7s - loss: 0.0490 - acc: 0.9905     
Epoch 9/10
9424/9424 [==============================] - 7s - loss: 0.0475 - acc: 0.9905     
Epoch 10/10
9424/9424 [==============================] - 7s - loss: 0.0469 - acc: 0.9905     


In [106]:
x = trainingX.iloc[7]
tx = [sequence.pad_sequences([x[:maxlen]], maxlen=maxlen)]
for i in np.random.randint(len(x), size=20):
    tx.append(sequence.pad_sequences([x[i: i + maxlen]], maxlen=maxlen))

In [107]:
np.where(model_lstm.predict_proba(np.vstack(tx)).mean(axis=0) > 0.045)

21/21 [==============================] - 0s


(array([ 41,  66,  76,  77,  78,  90, 120, 125, 136]),)

In [108]:
train_probs = model_lstm.predict_proba(x_train[:100])

100/100 [==============================] - 0s     


In [109]:
x_train[:100]

array([[44043, 30574, 48019, ..., 44213, 70192, 54583],
       [    0,     0,     0, ..., 19794,  2801,     1],
       [65504, 58152, 32046, ..., 23253, 36643,     1],
       ..., 
       [45154, 26871, 61827, ..., 37246,  2803,     1],
       [    0,     0,     0, ..., 31059, 73698,     1],
       [    0,     0,     0, ..., 32046, 17607,     1]], dtype=int32)

In [110]:
trainingY.head(100)

2014b_TrainingData_00001            [105, 66]
2014b_TrainingData_00024                 [41]
2014b_TrainingData_00041                [131]
2014b_TrainingData_00054                [125]
2014b_TrainingData_00135                 [56]
2014b_TrainingData_00143      [1, 76, 98, 28]
2014b_TrainingData_00173                  [2]
2014b_TrainingData_00175        [77, 120, 76]
2014b_TrainingData_00183                [118]
2014b_TrainingData_00204            [90, 142]
2014b_TrainingData_00206    [77, 136, 120, 2]
2014b_TrainingData_00236                 [76]
2014b_TrainingData_00240           [134, 117]
2014b_TrainingData_00244                [142]
2014b_TrainingData_00250                 [90]
2014b_TrainingData_00266            [41, 105]
2014b_TrainingData_00272                [134]
2014b_TrainingData_00277                 [90]
2014b_TrainingData_00278                 [70]
2014b_TrainingData_00282                [118]
2014b_TrainingData_00286                [120]
2014b_TrainingData_00290          

In [111]:
for i in train_probs:
    print np.mean(i), np.where(i > np.mean(i) + 3 * np.std(i))

0.0103271 (array([ 76,  77,  78,  90, 120, 125, 136]),)
0.00869188 (array([ 76,  77,  78,  90, 120, 125, 136]),)
0.00862108 (array([ 76,  77,  78,  90, 120, 125, 136]),)
0.00884387 (array([ 76,  77,  78,  90, 120, 125, 136]),)
0.00883101 (array([ 76,  77,  78,  90, 120, 125, 136]),)
0.00868308 (array([ 76,  77,  78,  90, 120, 125, 136]),)
0.00854737 (array([ 76,  77,  78,  90, 120, 125, 136]),)
0.00878521 (array([ 76,  77,  78,  90, 120, 125, 136]),)
0.010576 (array([ 76,  77,  78,  90, 120, 125, 136]),)
0.00858472 (array([ 76,  77,  78,  90, 120, 125, 136]),)
0.00869002 (array([ 76,  77,  78,  90, 120, 125, 136]),)
0.0105259 (array([ 76,  77,  78,  90, 120, 125, 136]),)
0.00889062 (array([ 76,  77,  78,  90, 120, 125, 136]),)
0.00881112 (array([ 76,  77,  78,  90, 120, 125, 136]),)
0.0129772 (array([ 76,  77,  78,  90, 120, 125, 136]),)
0.0112118 (array([ 76,  77,  78,  90, 120, 125, 136]),)
0.0089952 (array([ 76,  77,  78,  90, 120, 125, 136]),)
0.0162712 (array([ 76,  77,  78,  90, 

In [75]:
def get_classes(pred, scale_param=0.75, min_thresh=0.05):
    mx = pred.mean() + 3 * pred.std()
    return np.where(pred > mx)[0]  

In [18]:
with open('../data/TestData.json') as fl:
    data = json.load(fl)
    test_df = pd.DataFrame(data['TestData']).T
    del(data)

In [26]:
%%time
test_parsed_body = transform_text(test_df)
test_parsed_body_tokens = test_parsed_body.str.split(' ')
testX = test_parsed_body_tokens.map(lambda x: [word2ind.get(i, 0) for i in x])

CPU times: user 4.69 s, sys: 200 ms, total: 4.89 s
Wall time: 4.78 s


In [27]:
x_test = sequence.pad_sequences(testX, maxlen=maxlen)

In [29]:
test_probas = model_lstm.predict_proba(x_test)

7581/7581 [==============================] - 25s    


In [35]:
len(np.where(test_probas > 0.002)[1])

10968

(7581, 3)

In [76]:
%%time
# valid_test_feature_vec found below!

test_values = np.zeros([test_df.shape[0], len(topics)])
for ix, pred in enumerate(test_probas):
    for v in get_classes(pred):
        test_values[ix][v] = 1

test_sub_df = pd.DataFrame(test_values, columns=sorted(topics), index=test_df.index)
# for i in test_feature_vec[test_feature_vec.isnull()].index:
#     test_sub_df.ix[i] = np.zeros(len(topics))

CPU times: user 244 ms, sys: 4 ms, total: 248 ms
Wall time: 225 ms


In [77]:
test_sub_df[test_sub_df.sum(axis=1).isnull()]
test_sub_df.sum(axis=1).sum()

17838.0

In [74]:
test_sub_df.astype(int).reset_index().rename(columns={'index': 'id'}).sort_values('id').to_csv('lstm.2014b_training_700_maxlen_64cell_100epochs_0.0025_threshold.csv', index=False)